In [13]:
pip install keras_metrics

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [14]:
import numpy as np
from tensorflow import keras
import keras_metrics
import pandas as pd
from tensorflow.keras import regularizers 

In [15]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open('min-ind.txt', "r", encoding="utf-8") as f:
  lines = f.read().split("\n")
  #print('lines',lines)
#print('lines',lines[:len(lines)])
print(len(lines))
for line in lines:
  if len(line.split(','))==2:
    input_text, target_text = line.split(",")
  # We use "tab" as the "start sequence" character
  # for the targets, and "\n" as "end sequence" character.
  target_text = "\t" + target_text + "\n"
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
      if char not in input_characters:
          input_characters.add(char)
  for char in target_text:
      if char not in target_characters:
          target_characters.add(char)

13762


In [16]:
for i,(train_index,val_index) in enumerate(kf.split(np.zeros(len(encoder_input_data_k)))):
  print('Iteration number : ', i+1)

  encoder_input_data_t = pd.Series(list(encoder_input_data_k)).iloc[train_index].to_numpy()
  decoder_input_data_t = pd.Series(list(decoder_input_data_k)).iloc[train_index].to_numpy()
  decoder_target_data_t = pd.Series(list(decoder_target_data_k)).iloc[train_index].to_numpy()

  encoder_input_data_val = pd.Series(list(encoder_input_data_k)).iloc[val_index].to_numpy()
  decoder_input_data_val = pd.Series(list(decoder_input_data_k)).iloc[val_index].to_numpy()
  decoder_target_data_val = pd.Series(list(decoder_target_data_k)).iloc[val_index].to_numpy()

  encoder_input_data_t = np.array([list(en) for en in encoder_input_data_t])
  decoder_input_data_t = np.array([list(en) for en in decoder_input_data_t])
  decoder_target_data_t = np.array([list(en) for en in decoder_target_data_t])

  encoder_input_data_val = np.array([list(en) for en in encoder_input_data_val])
  decoder_input_data_val = np.array([list(en) for en in decoder_input_data_val])
  decoder_target_data_val = np.array([list(en) for en in decoder_target_data_val])

  x_train = [encoder_input_data_t, decoder_input_data_t]
  y_train = decoder_target_data_t

  x_val = [encoder_input_data_val,decoder_input_data_val]
  y_val = decoder_target_data_val

  encoder_inputs = keras.Input(shape=(None,num_encoder_tokens))
  encoder_outputs,states_h,states_c=keras.layers.LSTM(latent_dim_2, return_state=True, kernel_regularizer=regularizers.l2(l2=1e-5),
    bias_regularizer=regularizers.l2(1e-5))(encoder_inputs)
  #encoder_outputs,states_h,states_c=keras.layers.LSTM(latent_dim_2, return_state=True)(encoder_inputs)

  # We discard `encoder_outputs` and only keep the states.
  #state_h = keras.layers.Concatenate()([forward_h,forward_c])
  #state_c = keras.layers.Concatenate()([backward_h,backward_c])
  encoder_states = [states_h,states_c] # hidden and cell state 

  # Set up the decoder, using `encoder_states` as initial state.
  decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
  # We set up our decoder to return full output sequences,
  # and to return internal states as well. We don't use the
  # return states in the training model, but we will use them in inference.
  decoder_outputs,*decoder_states= keras.layers.LSTM(latent_dim_3, 
                                                    return_sequences=True, 
                                                    return_state=True, kernel_regularizer=regularizers.l2(l2=1e-5),
    bias_regularizer=regularizers.l2(1e-5))(decoder_inputs, initial_state=encoder_states)
  #decoder_outputs,*decoder_states= keras.layers.LSTM(latent_dim_3, return_sequences=True, 
                                                    #return_state=True)(decoder_inputs, initial_state=encoder_states)

  decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
  #decoder_dropout = keras.layers.Dropout(0.8)
  decoder_outputs = decoder_dense(decoder_outputs)
  #decoder_outputs = decoder_dropout(decoder_outputs)
  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

  # Compile Model
  model.compile(
      optimizer="adam", loss="categorical_crossentropy", 
      metrics=["accuracy",keras.metrics.Recall(),keras.metrics.Precision()]
  )
  # Fitting Model
  model_history = model.fit(
      x_train, y_train,
      batch_size=batch_size,
      epochs=epochs,
      validation_data = (x_val,y_val)
  )

  # Save model
  model_name = 'model_lstm_256_{}-Fold-{}'.format(fold_var,i+1)
  model.save(model_name)

  # Define sampling models
  # Restore the model and construct the encoder and decoder.
  model_inf = keras.models.load_model(model_name)
  encoder_inputs = model_inf.input[0]  # input_1
  encoder_outputs,states_h,states_c  = model_inf.layers[2].output  # lstm_1
  #states_h = keras.layers.Concatenate()([state_h_enc,state_c_enc])
  #states_c = keras.layers.Concatenate()([backward_h,backward_c])
  encoder_states = [states_h,states_c]
  encoder_model = keras.Model(encoder_inputs, encoder_states)

  decoder_inputs = model_inf.input[1]  # input_2
  decoder_state_input_h = keras.Input(shape=(latent_dim_3,), name="input_3")
  decoder_state_input_c = keras.Input(shape=(latent_dim_3,), name="input_5")
  decoder_states_inputs = [decoder_state_input_h,decoder_state_input_c]
  decoder_lstm = model_inf.layers[3]
  decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
      decoder_inputs, initial_state=decoder_states_inputs
  )
  decoder_states = [state_h_dec, state_c_dec]
  decoder_dense = model_inf.layers[4]
  decoder_outputs = decoder_dense(decoder_outputs)
  #decoder_dropout = model_inf.layers[7]
  #decoder_outputs = decoder_dropout(decoder_outputs)
  decoder_model = keras.Model(
      [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
  )
  # Reverse-lookup token index to decode sequences back to
  # something readable.
  reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
  reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

  def decode_sequence(input_seq):
      # Encode the input as state vectors.
      states_value = encoder_model.predict(input_seq)

      # Generate empty target sequence of length 1.
      target_seq = np.zeros((1, 1, num_decoder_tokens))
      # Populate the first character of target sequence with the start character.
      target_seq[0, 0, target_token_index["\t"]] = 1.0

      # Sampling loop for a batch of sequences
      # (to simplify, here we assume a batch of size 1).
      stop_condition = False
      decoded_sentence = ""
      while not stop_condition:
          output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

          # Sample a token

          sampled_token_index = np.argmax(output_tokens[0, -1, :])
          sampled_char = reverse_target_char_index[sampled_token_index]
          decoded_sentence += sampled_char

          # Exit condition: either hit max length
          # or find stop character.
          if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
              stop_condition = True

          # Update the target sequence (of length 1).
          target_seq = np.zeros((1, 1, num_decoder_tokens))
          target_seq[0, 0, sampled_token_index] = 1.0

          # Update states
          states_value = [h, c]
      return decoded_sentence

  # Evaluate Model
  results = model.evaluate(x_val, y_val, batch_size=batch_size)
  results = dict(zip(model.metrics_names,results))
  
  # Save Evaluation Result
  validation_accuracy.append(results['accuracy'])
  validation_loss.append(results['loss'])
  
 
  # Check Result Translation
  
  # Check Result Translation
  match = 0
  for seq_index in val_index:
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)[1:-1]
    #print("-",seq_index)
    #print("Input sentence:", lines_1[seq_index])
    #print("Decoded sentence:", decoded_sentence)
    #print("Target sentence:", lines_2[seq_index][:-1])
    exact_result = decoded_sentence == lines_2[seq_index][:-1]
   #print("Exact Match:", str(exact_result))
    match += exact_result
  print("Exact Match count : ", match)
  print("Tested Data count : ", len(val_index))
  print("Translation Accuracy : ", match/len(val_index))
  

Iteration number :  1


InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

In [ ]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [ ]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 13762
Number of unique input tokens: 35
Number of unique output tokens: 32
Max sequence length for inputs: 21
Max sequence length for outputs: 19


In [ ]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])
 
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
seed = 7
np.random.seed(seed)
kfold_index,test_index = train_test_split(list(range(len(encoder_input_data))),test_size=0.2,random_state=seed)
fold_var = 5
kf = KFold(n_splits = fold_var,random_state=seed, shuffle=True)
validation_accuracy = []
validation_loss = []
exac_match_accuracy = []

In [ ]:
encoder_input_data_k = pd.Series(list(encoder_input_data)).iloc[kfold_index].to_numpy()
decoder_input_data_k = pd.Series(list(decoder_input_data)).iloc[kfold_index].to_numpy()
decoder_target_data_k = pd.Series(list(decoder_target_data)).iloc[kfold_index].to_numpy()

encoder_input_data_k = np.array([list(en) for en in encoder_input_data_k])
decoder_input_data_k = np.array([list(en) for en in decoder_input_data_k])
decoder_target_data_k = np.array([list(en) for en in decoder_target_data_k])

encoder_input_data_test = pd.Series(list(encoder_input_data)).iloc[test_index].to_numpy()
decoder_input_data_test = pd.Series(list(decoder_input_data)).iloc[test_index].to_numpy()
decoder_target_data_test = pd.Series(list(decoder_target_data)).iloc[test_index].to_numpy()

encoder_input_data_test = np.array([list(en) for en in encoder_input_data_test])
decoder_input_data_test = np.array([list(en) for en in decoder_input_data_test])
decoder_target_data_test = np.array([list(en) for en in decoder_target_data_test])

In [ ]:
batch_size = 64  # Batch size for training.
epochs = 25  # Number of epochs to train for.
latent_dim_1 = 128
latent_dim_2 = 512
latent_dim_3 = 512 # Latent dimensionality of the encoding space.
num_samples = len(encoder_input_data)  # Number of samples to train on.

In [ ]:
for i,(train_index,val_index) in enumerate(kf.split(np.zeros(len(encoder_input_data_k)))):
  print('Iteration number : ', i+1)

  encoder_input_data_t = pd.Series(list(encoder_input_data_k)).iloc[train_index].to_numpy()
  decoder_input_data_t = pd.Series(list(decoder_input_data_k)).iloc[train_index].to_numpy()
  decoder_target_data_t = pd.Series(list(decoder_target_data_k)).iloc[train_index].to_numpy()

  encoder_input_data_val = pd.Series(list(encoder_input_data_k)).iloc[val_index].to_numpy()
  decoder_input_data_val = pd.Series(list(decoder_input_data_k)).iloc[val_index].to_numpy()
  decoder_target_data_val = pd.Series(list(decoder_target_data_k)).iloc[val_index].to_numpy()

  encoder_input_data_t = np.array([list(en) for en in encoder_input_data_t])
  decoder_input_data_t = np.array([list(en) for en in decoder_input_data_t])
  decoder_target_data_t = np.array([list(en) for en in decoder_target_data_t])

  encoder_input_data_val = np.array([list(en) for en in encoder_input_data_val])
  decoder_input_data_val = np.array([list(en) for en in decoder_input_data_val])
  decoder_target_data_val = np.array([list(en) for en in decoder_target_data_val])

  x_train = [encoder_input_data_t, decoder_input_data_t]
  y_train = decoder_target_data_t

  x_val = [encoder_input_data_val,decoder_input_data_val]
  y_val = decoder_target_data_val

  encoder_inputs = keras.Input(shape=(None,num_encoder_tokens))
  encoder_outputs,states_h,states_c=keras.layers.LSTM(latent_dim_2, return_state=True, kernel_regularizer=regularizers.l2(l2=1e-5),
    bias_regularizer=regularizers.l2(1e-5))(encoder_inputs)
  #encoder_outputs,states_h,states_c=keras.layers.LSTM(latent_dim_2, return_state=True)(encoder_inputs)

  # We discard `encoder_outputs` and only keep the states.
  #state_h = keras.layers.Concatenate()([forward_h,forward_c])
  #state_c = keras.layers.Concatenate()([backward_h,backward_c])
  encoder_states = [states_h,states_c] # hidden and cell state 

  # Set up the decoder, using `encoder_states` as initial state.
  decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
  # We set up our decoder to return full output sequences,
  # and to return internal states as well. We don't use the
  # return states in the training model, but we will use them in inference.
  decoder_outputs,*decoder_states= keras.layers.LSTM(latent_dim_3, 
                                                    return_sequences=True, 
                                                    return_state=True, kernel_regularizer=regularizers.l2(l2=1e-5),
    bias_regularizer=regularizers.l2(1e-5))(decoder_inputs, initial_state=encoder_states)
  #decoder_outputs,*decoder_states= keras.layers.LSTM(latent_dim_3, return_sequences=True, 
                                                    #return_state=True)(decoder_inputs, initial_state=encoder_states)

  decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
  #decoder_dropout = keras.layers.Dropout(0.8)
  decoder_outputs = decoder_dense(decoder_outputs)
  #decoder_outputs = decoder_dropout(decoder_outputs)
  # Define the model that will turn
  # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
  model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

  # Compile Model
  model.compile(
      optimizer="adam", loss="categorical_crossentropy", 
      metrics=["accuracy",keras.metrics.Recall(),keras.metrics.Precision()]
  )
  # Fitting Model
  model_history = model.fit(
      x_train, y_train,
      batch_size=batch_size,
      epochs=epochs,
      validation_data = (x_val,y_val)
  )

  # Save model
  model_name = 'model_lstm_256_{}-Fold-{}'.format(fold_var,i+1)
  model.save(model_name)

  # Define sampling models
  # Restore the model and construct the encoder and decoder.
  model_inf = keras.models.load_model(model_name)
  encoder_inputs = model_inf.input[0]  # input_1
  encoder_outputs,states_h,states_c  = model_inf.layers[2].output  # lstm_1
  #states_h = keras.layers.Concatenate()([state_h_enc,state_c_enc])
  #states_c = keras.layers.Concatenate()([backward_h,backward_c])
  encoder_states = [states_h,states_c]
  encoder_model = keras.Model(encoder_inputs, encoder_states)

  decoder_inputs = model_inf.input[1]  # input_2
  decoder_state_input_h = keras.Input(shape=(latent_dim_3,), name="input_3")
  decoder_state_input_c = keras.Input(shape=(latent_dim_3,), name="input_5")
  decoder_states_inputs = [decoder_state_input_h,decoder_state_input_c]
  decoder_lstm = model_inf.layers[3]
  decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
      decoder_inputs, initial_state=decoder_states_inputs
  )
  decoder_states = [state_h_dec, state_c_dec]
  decoder_dense = model_inf.layers[4]
  decoder_outputs = decoder_dense(decoder_outputs)
  #decoder_dropout = model_inf.layers[7]
  #decoder_outputs = decoder_dropout(decoder_outputs)
  decoder_model = keras.Model(
      [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
  )
  # Reverse-lookup token index to decode sequences back to
  # something readable.
  reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
  reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

  def decode_sequence(input_seq):
      # Encode the input as state vectors.
      states_value = encoder_model.predict(input_seq)

      # Generate empty target sequence of length 1.
      target_seq = np.zeros((1, 1, num_decoder_tokens))
      # Populate the first character of target sequence with the start character.
      target_seq[0, 0, target_token_index["\t"]] = 1.0

      # Sampling loop for a batch of sequences
      # (to simplify, here we assume a batch of size 1).
      stop_condition = False
      decoded_sentence = ""
      while not stop_condition:
          output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

          # Sample a token

          sampled_token_index = np.argmax(output_tokens[0, -1, :])
          sampled_char = reverse_target_char_index[sampled_token_index]
          decoded_sentence += sampled_char

          # Exit condition: either hit max length
          # or find stop character.
          if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
              stop_condition = True

          # Update the target sequence (of length 1).
          target_seq = np.zeros((1, 1, num_decoder_tokens))
          target_seq[0, 0, sampled_token_index] = 1.0

          # Update states
          states_value = [h, c]
      return decoded_sentence

  # Evaluate Model
  results = model.evaluate(x_val, y_val, batch_size=batch_size)
  results = dict(zip(model.metrics_names,results))
  
  # Save Evaluation Result
  validation_accuracy.append(results['accuracy'])
  validation_loss.append(results['loss'])
  
 
  # Check Result Translation
  
  # Check Result Translation
  list_match = []
  count = 0
  print("Checking Translation Accuracy")
  for seq_index in val_index:
      # Take one sequence (part of the training set)
      # for trying out decoding.
      input_seq = encoder_input_data[seq_index : seq_index + 1]
      decoded_sentence = decode_sequence(input_seq)
      decoded_sentence = decoded_sentence[:-1]
      # print("-",seq_index)
      # print("Input sentence:", ''.join(input_texts[seq_index]))
      # print("Decoded sentence:", ''.join(decoded_sentence))
      # print("Target sentence:", ''.join(target_texts[seq_index][1:-1]))
      # print("Exact Match:", ''.join(str(exact_result)))
      exact_result = decoded_sentence==target_texts[seq_index][1:-1]
      list_match.append(exact_result)
      count+=1
  print("Exact Match count : ", sum(list_match))
  print("Tested Data count : ", count)
  print("Translation Accuracy : ", sum(list_match)/count)
  exac_match_accuracy.append(sum(list_match)/count)
  

Iteration number :  1
Epoch 1/25
138/138 [==============================] - 103s 714ms/step - loss: 1.3424 - accuracy: 0.6280 - recall: 0.5364 - precision: 0.9682 - val_loss: 1.0667 - val_accuracy: 0.6970 - val_recall: 0.5855 - val_precision: 0.9876
Epoch 2/25
138/138 [==============================] - 99s 716ms/step - loss: 0.9174 - accuracy: 0.7291 - recall: 0.6310 - precision: 0.9695 - val_loss: 0.8747 - val_accuracy: 0.7392 - val_recall: 0.6418 - val_precision: 0.9632
Epoch 3/25
138/138 [==============================] - 101s 729ms/step - loss: 0.8080 - accuracy: 0.7573 - recall: 0.6686 - precision: 0.9569 - val_loss: 0.8126 - val_accuracy: 0.7578 - val_recall: 0.6827 - val_precision: 0.9320
Epoch 4/25
138/138 [==============================] - 103s 747ms/step - loss: 0.7475 - accuracy: 0.7747 - recall: 0.6889 - precision: 0.9496 - val_loss: 0.7341 - val_accuracy: 0.7792 - val_recall: 0.6910 - val_precision: 0.9482
Epoch 5/25
138/138 [==============================] - 102s 743ms/st

INFO:tensorflow:Assets written to: model_lstm_256_5-Fold-1/assets


INFO:tensorflow:Assets written to: model_lstm_256_5-Fold-1/assets


35/35 [==============================] - 6s 177ms/step - loss: 0.3484 - accuracy: 0.9217 - recall: 0.9135 - precision: 0.9365
Checking Translation Accuracy
Exact Match count :  607
Tested Data count :  2202
Translation Accuracy :  0.2756584922797457
Iteration number :  2
Epoch 1/25
138/138 [==============================] - 109s 759ms/step - loss: 1.3240 - accuracy: 0.6355 - recall_1: 0.5422 - precision_1: 0.9693 - val_loss: 1.0393 - val_accuracy: 0.7020 - val_recall_1: 0.5808 - val_precision_1: 0.9933
Epoch 2/25
138/138 [==============================] - 104s 751ms/step - loss: 0.9264 - accuracy: 0.7286 - recall_1: 0.6272 - precision_1: 0.9678 - val_loss: 0.8501 - val_accuracy: 0.7503 - val_recall_1: 0.6424 - val_precision_1: 0.9765
Epoch 3/25
138/138 [==============================] - 103s 748ms/step - loss: 0.8057 - accuracy: 0.7588 - recall_1: 0.6652 - precision_1: 0.9567 - val_loss: 0.7674 - val_accuracy: 0.7675 - val_recall_1: 0.6799 - val_precision_1: 0.9510
Epoch 4/25
138/138 [

INFO:tensorflow:Assets written to: model_lstm_256_5-Fold-2/assets


...

INFO:tensorflow:Assets written to: model_lstm_256_5-Fold-4/assets


35/35 [==============================] - 6s 176ms/step - loss: 0.3219 - accuracy: 0.9246 - recall_3: 0.9150 - precision_3: 0.9415
Checking Translation Accuracy
Exact Match count :  738
Tested Data count :  2202
Translation Accuracy :  0.335149863760218
Iteration number :  5
Epoch 1/25
138/138 [==============================] - 106s 738ms/step - loss: 1.3218 - accuracy: 0.6371 - recall_4: 0.5401 - precision_4: 0.9677 - val_loss: 1.0387 - val_accuracy: 0.7055 - val_recall_4: 0.5965 - val_precision_4: 0.9817
Epoch 2/25
138/138 [==============================] - 101s 729ms/step - loss: 0.9200 - accuracy: 0.7289 - recall_4: 0.6300 - precision_4: 0.9696 - val_loss: 0.8535 - val_accuracy: 0.7389 - val_recall_4: 0.6436 - val_precision_4: 0.9636
Epoch 3/25
138/138 [==============================] - 101s 730ms/step - loss: 0.8107 - accuracy: 0.7561 - recall_4: 0.6653 - precision_4: 0.9581 - val_loss: 0.7789 - val_accuracy: 0.7656 - val_recall_4: 0.6739 - val_precision_4: 0.9538
Epoch 4/25
138/13

INFO:tensorflow:Assets written to: model_lstm_256_5-Fold-5/assets


INFO:tensorflow:Assets written to: model_lstm_256_5-Fold-5/assets


35/35 [==============================] - 6s 180ms/step - loss: 0.3393 - accuracy: 0.9215 - recall_4: 0.9115 - precision_4: 0.9391
Checking Translation Accuracy
Exact Match count :  722
Tested Data count :  2201
Translation Accuracy :  0.328032712403453


In [ ]:
# Show Accuracy Result
for i,val_acc in enumerate(validation_accuracy):
  print("Fold {} Accuracy : {}".format(i+1,val_acc))
print("Average Accuracy : %.2f%% (+/- %.2f%%)" % (np.mean(validation_accuracy), np.std(validation_accuracy)))

Fold 1 Accuracy : 0.9216979742050171
Fold 2 Accuracy : 0.9274582862854004
Fold 3 Accuracy : 0.9251159429550171
Fold 4 Accuracy : 0.9246140122413635
Fold 5 Accuracy : 0.9215428233146667
Average Accuracy : 0.92% (+/- 0.00%)


In [ ]:
#Prep Test Data
x_test = [encoder_input_data_test,decoder_input_data_test]
y_test = decoder_target_data_test

# Evaluate Model
results = model.evaluate(x_test, y_test, batch_size=batch_size)
results = dict(zip(model.metrics_names,results))

44/44 [==============================] - 8s 191ms/step - loss: 0.3559 - accuracy: 0.9166 - recall_4: 0.9065 - precision_4: 0.9342


In [ ]:
# Check Result Translation
list_match = []
count = 0
print("Checking Translation Accuracy")
for seq_index in test_index:
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    decoded_sentence = decoded_sentence[:-1]
    print("-",seq_index)
    print("Input sentence:", ''.join(input_texts[seq_index]))
    print("Decoded sentence:", ''.join(decoded_sentence))
    print("Target sentence:", ''.join(target_texts[seq_index][1:-1]))
    exact_result = decoded_sentence==target_texts[seq_index][1:-1]
    print("Exact Match:", ''.join(str(exact_result)))
   
    list_match.append(exact_result)
    count+=1
print("Exact Match count : ", sum(list_match))
print("Tested Data count : ", count)
print("Translation Accuracy : ", sum(list_match)/count)

Streaming output truncated to the last 5000 lines.
Target sentence: rencananya
Exact Match: True
- 7363
Input sentence: bang
Decoded sentence: banginan
Target sentence: bang
Exact Match: False
- 4670
Input sentence: kontribusi
Decoded sentence: kontrosisi
Target sentence: kontribusi
Exact Match: False
- 4532
Input sentence: parawatan
Decoded sentence: perawatanawanatakan
Target sentence: perawatan
Exact Match: False
- 8655
Input sentence: perindustrian
Decoded sentence: perindustikan
Target sentence: perindustrian
Exact Match: False
- 2977
Input sentence: kuciang
Decoded sentence: kucing
Target sentence: kucing
Exact Match: True
- 13378
Input sentence: abih-abihan
Decoded sentence: hamput
Target sentence: radikal
Exact Match: False
- 11945
Input sentence: carito
Decoded sentence: cerita
Target sentence: cerita
Exact Match: True
- 10890
Input sentence: kabodohan
Decoded sentence: kebodohannya
Target sentence: kebodohan
Exact Match: False
- 2859
Input sentence: sandiwara
Decoded sentence

In [ ]:
!zip -r model_lstm_256_5-Fold-3.zip model_lstm_256_5-Fold-3

  adding: model_lstm_256_5-Fold-3/ (stored 0%)
  adding: model_lstm_256_5-Fold-3/assets/ (stored 0%)
  adding: model_lstm_256_5-Fold-3/keras_metadata.pb (deflated 90%)
  adding: model_lstm_256_5-Fold-3/saved_model.pb (deflated 90%)
  adding: model_lstm_256_5-Fold-3/variables/ (stored 0%)
  adding: model_lstm_256_5-Fold-3/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: model_lstm_256_5-Fold-3/variables/variables.index (deflated 65%)
